In [0]:
import torch
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
import numpy as np
np.random.seed(0)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [0]:
from torchvision import datasets, transforms
import torch.nn.functional as F
from torch import nn


In [0]:
mean, std = (0.5,) , (0.5,)
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean, std)])
#download dataset
trainset = datasets.FashionMNIST('~/.pytorch/FMNIST/', download = True, train = True, transform = transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = 64, shuffle = True)

testset = datasets.FashionMNIST('~/.pytorch/FMNIST/', download = True, train = False, transform = transform)
testloader = torch.utils.data.DataLoader(testset, batch_size = 64, shuffle = False)


In [0]:
#define network
class FMNIST(nn.Module):
  def __init__(self):
    super().__init__()
    self.fc1 = nn.Linear(784, 128)
    self.fc2 = nn.Linear(128, 64)
    self.fc3 = nn.Linear(64, 10)

  def forward(self, x):
    x = x.view(x.shape[0], -1)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    x = F.log_softmax(x, dim =1)

model = FMNIST()
model.to(device)

FMNIST(
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=10, bias=True)
)

In [0]:
#Loss, Autograd, and Optimizers
from torch import optim

criterion = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr = 0.01)

num_epochs = 3
train_tracker, test_tracker, accuracy_tracker =[], [], []
for i in range(num_epochs):
  cum_loss = 0
  for batch, (images, labels) in enumerate(trainloader,1):
    images = images.to(device)
    labels = labels.to(device)
    optimizer.zero_grad()
    output = model(images)
    loss = criterion(output, labels)
    loss.backward()
    optimizer.step()

    cum_loss += loss.item()
  
  train_tracker.appen(cum_loss/len(trainloader))
  #print(f"Training loss: {cum_loss/len(trainloader)}")
  print(f"Epoch({i+1}/{num_epochs}) | Training loss: {cum_loss/len(trainloader)} |", end ='')

  test_loss = 0
  num_correct = 0
  total = 0
  for batch, (images, labels) in enumerate(testloader, 1):
    images = images.to(device)
    labels = images.to(device)
    logps = model(images)
    batch_loss = criterion(logps, labels)
    test_loss += batch_loss.item()
    output = torch.exp(logps)
    pred = torch.argmax(output, 1)
    total += labels.size(0)
    num_correct += (pred == labels).sum().item()
    
    test_tracker.append(test_loss/len(testloader))
    print(f"Test Loss : {test_loss/len(testloader)} |", end = '')
    accuracy_tracker.append(num_correct/total)
    print(f'Accuracy: {num_correct/total}')
  print(f'\nNumber Correct : {num_correct}, Total: {total}')
  print(f'Accuracy of the model after 30 epochs on 10000 test images: {num_correct*100/total}%')


In [0]:
import matplotlib.pyplot as plt
plt.plot(train_tracker, label = 'training_loss')
plt.plot(test_tracker, label = 'testing_loss')
plt.plot(accuracy_tracker, label = 'test_accuracy')
plt.legend()

In [0]:
#testing phase
import matplotlib.pyplot as plt
images, labels = next(iter(testloader))

test_image_id = 15
img = images[test_image_id].view(1,784)

model.to('cpu')
with torch.no_grad():
  logps = model(img)

In [0]:
#ps = torch.exp(logps)
#nps = ps.numpy()[0]

In [0]:
FMNIST_labels = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Sport Shoes', 'Bag', 'Ankle Boot', 'Sneaker']
plt.xticks(np.arrange(10), labels = FMNIST_labels, rotation = 'vertical')
plt.bar(np.arrange(10), nps)

In [0]:
def normalize(tensor):
  tensor = tensor*0.5 + 0.5
  return tensor

img = img.view(28, -1)
img = denormalize(img)
plt.imshow(img, cmap = 'gray')